In [ ]:
#RANDOM FOREST MODEL WITH PHYSICOCHEMICAL FEATURES
# 1) Importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
!pip install Bio
from Bio.SeqUtils.ProtParam import ProteinAnalysis

# 2) Load the dataset and define stability classes
def categorize_half_life(half_life):
    if half_life < 12:
        return "Unstable"
    elif half_life < 2500:
        return "Moderately Stable"
    else:
        return "Highly stable"

df = pd.read_csv('/content/pepdist_final.csv')
df['stability_class'] = df['half_life'].apply(categorize_half_life)

# 3) Enhanced Feature Engineering Functions
amino_acids = list('ACDEFGHIKLMNPQRSTVWY')

# Hydrophobicity scales (Kyte-Doolittle scale)
hydrophobicity_scale = {
    'A': 1.8, 'C': 2.5, 'D': -3.5, 'E': -3.5, 'F': 2.8,
    'G': -0.4, 'H': -3.2, 'I': 4.5, 'K': -3.9, 'L': 3.8,
    'M': 1.9, 'N': -3.5, 'P': -1.6, 'Q': -3.5, 'R': -4.5,
    'S': -0.8, 'T': -0.7, 'V': 4.2, 'W': -0.9, 'Y': -1.3
}

def peptide_features(seq):
    # Amino acid composition
    aa_counts = [seq.count(aa) for aa in amino_acids]
    
    # Physicochemical properties using BioPython
    analysis = ProteinAnalysis(seq)
    
    # Molecular weight
    molecular_weight = analysis.molecular_weight()
    
    # Charge at pH 7.0
    charge = analysis.charge_at_pH(7.0)
    
    # Hydrophobicity (average Kyte-Doolittle score)
    hydrophobicity = sum(hydrophobicity_scale.get(aa, 0) for aa in seq) / len(seq) if len(seq) > 0 else 0
    
    # Additional BioPython features
    gravy = analysis.gravy()  # Grand average of hydropathy
    instability_index = analysis.instability_index()
    isoelectric_point = analysis.isoelectric_point()
    
    # Combine all features
    features = aa_counts + [molecular_weight, charge, hydrophobicity, gravy, instability_index, isoelectric_point]
    
    return features

# Create feature matrix
peptide_feature_matrix = np.array([peptide_features(seq) for seq in df['peptide_seq']])

# Create feature names
feature_names = amino_acids + ['molecular_weight', 'charge_pH7', 'hydrophobicity_KD', 'gravy', 'instability_index', 'isoelectric_point']

features_df = pd.DataFrame(peptide_feature_matrix, columns=feature_names)
features_df['stability_class'] = df['stability_class']

print("Feature matrix shape:", features_df.shape)
print("Features included:", feature_names)

# 4) Prepare for training
X = features_df.drop('stability_class', axis=1)
y = features_df['stability_class']
peptide_seqs = df['peptide_seq'].to_numpy()

# 5) K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
overall_conf_matrix = np.zeros((3, 3), dtype=int)
class_names = ["Highly stable", "Moderately Stable", "Unstable"]

all_true = []
all_pred = []
all_scores = []
peptides_used = []

# We will track probability of "Moderately Stable"
target_class = "Moderately Stable"

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    proba = model.predict_proba(X_test)

    # Get probability for the chosen class
    class_index = list(model.classes_).index(target_class)
    scores = proba[:, class_index]

    all_true.extend(y_test)
    all_pred.extend(y_pred)
    all_scores.extend(scores)
    peptides_used.extend(peptide_seqs[test_index])

    cm = confusion_matrix(y_test, y_pred, labels=class_names)
    overall_conf_matrix += cm

# 6) Classification report and confusion matrix
print("\nOverall Classification Report:")
print(classification_report(all_true, all_pred, labels=class_names))

plt.figure(figsize=(8, 6))
sns.heatmap(overall_conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Random Forest Confusion Matrix (Aggregated from 5-Fold Cross-Validation)')
plt.tight_layout()
plt.show()

# 7) Save enhanced results to CSV
rf_pred_df = pd.DataFrame({
    'peptide_sequences': peptides_used,
    'predicted_stability': all_pred,
    'rf_score': all_scores
})

# Add physicochemical properties for each peptide
physico_props = []
for seq in peptides_used:
    analysis = ProteinAnalysis(seq)
    props = {
        'molecular_weight': analysis.molecular_weight(),
        'charge_pH7': analysis.charge_at_pH(7.0),
        'hydrophobicity_KD': sum(hydrophobicity_scale.get(aa, 0) for aa in seq) / len(seq),
        'gravy': analysis.gravy(),
        'instability_index': analysis.instability_index(),
        'isoelectric_point': analysis.isoelectric_point()
    }
    physico_props.append(props)

physico_df = pd.DataFrame(physico_props)
rf_pred_df = pd.concat([rf_pred_df, physico_df], axis=1)

rf_pred_df.to_csv("rf_predictions.csv", index=False)
print("Random Forest predictions with physicochemical properties saved to rf_predictions.csv")

# 8) ROC Curve Analysis
from sklearn.metrics import roc_curve, auc

# Binary label oluştur: Stable (>=12) -> 1, Unstable (<12) -> 0
df['binary_label'] = df['half_life'].apply(lambda x: 1 if x >= 12 else 0)

# ROC eğrisi için tahmin edilen peptitlere karşılık gelen gerçek binary label'ları al
true_labels_binary = df.set_index('peptide_seq').loc[peptides_used, 'binary_label'].values

# ROC curve ve AUROC hesapla
fpr, tpr, _ = roc_curve(true_labels_binary, all_scores)
roc_auc = auc(fpr, tpr)

# ROC eğrisini çiz
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='royalblue', lw=2, label=f' Random Forest (AUROC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(' Random Forest ROC Curve')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nModel Performance Summary:")
print(f"AUROC Score: {roc_auc:.3f}")



In [ ]:
#DEEP LEARNING MODEL WITH PHYSICOCHEMICAL FEATURES
# 1) Importing libraries
import pandas as pd
import numpy as np
import random, os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
!pip install Bio
from Bio.SeqUtils.ProtParam import ProteinAnalysis

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

def get_class_weights(y):
    class_counts = np.bincount(y)
    total = len(y)
    return torch.tensor([total / c for c in class_counts], dtype=torch.float)

seed_everything(42)

# 2) Load and label dataset
df = pd.read_csv("/content/pepdist_final.csv")

def categorize_half_life(h):
    return 0 if h < 12 else (1 if h < 2500 else 2)

df["stability_class"] = df["half_life"].apply(categorize_half_life)

peptides = df["peptide_seq"].values
y = df["stability_class"].values
dist_cols = [c for c in df.columns if c.startswith("dist_")]
dist_mat = df[dist_cols].values.astype(np.float32)

# 3) One-hot encode peptides and extract physicochemical features
AA_ORDER = "ACDEFGHIKLMNPQRSTVWY"
AA_TO_IDX = {aa: i for i, aa in enumerate(AA_ORDER)}

# Hydrophobicity scales (Kyte-Doolittle scale)
hydrophobicity_scale = {
    'A': 1.8, 'C': 2.5, 'D': -3.5, 'E': -3.5, 'F': 2.8,
    'G': -0.4, 'H': -3.2, 'I': 4.5, 'K': -3.9, 'L': 3.8,
    'M': 1.9, 'N': -3.5, 'P': -1.6, 'Q': -3.5, 'R': -4.5,
    'S': -0.8, 'T': -0.7, 'V': 4.2, 'W': -0.9, 'Y': -1.3
}

def one_hot(seq):
    enc = np.zeros((len(seq), len(AA_ORDER)), dtype=np.float32)
    for i, aa in enumerate(seq):
        if aa in AA_TO_IDX:
            enc[i, AA_TO_IDX[aa]] = 1.0
    return enc.flatten()

def extract_physicochemical_features(seq):
    """Extract physicochemical features from peptide sequence"""
    analysis = ProteinAnalysis(seq)
    
    # Molecular weight
    molecular_weight = analysis.molecular_weight()
    
    # Charge at pH 7.0
    charge = analysis.charge_at_pH(7.0)
    
    # Hydrophobicity (average Kyte-Doolittle score)
    hydrophobicity = sum(hydrophobicity_scale.get(aa, 0) for aa in seq) / len(seq) if len(seq) > 0 else 0
    
    # Additional BioPython features
    gravy = analysis.gravy()  # Grand average of hydropathy
    instability_index = analysis.instability_index()
    isoelectric_point = analysis.isoelectric_point()
    
    return np.array([molecular_weight, charge, hydrophobicity, gravy, instability_index, isoelectric_point], dtype=np.float32)

# Extract features for all peptides
pept_mat = np.stack([one_hot(s) for s in peptides])
physico_mat = np.stack([extract_physicochemical_features(s) for s in peptides])

print(f"One-hot encoded peptide matrix shape: {pept_mat.shape}")
print(f"Physicochemical features matrix shape: {physico_mat.shape}")

# 4) Train-test split
X_pep_tr, X_pep_te, X_physico_tr_raw, X_physico_te_raw, X_dist_tr_raw, X_dist_te_raw, y_tr, y_te = train_test_split(
    pept_mat, physico_mat, dist_mat, y, test_size=0.20, stratify=y, random_state=42)

# 5) Scale distance and physicochemical features
dist_scaler = StandardScaler()
X_dist_tr = dist_scaler.fit_transform(X_dist_tr_raw)
X_dist_te = dist_scaler.transform(X_dist_te_raw)

physico_scaler = StandardScaler()
X_physico_tr = physico_scaler.fit_transform(X_physico_tr_raw)
X_physico_te = physico_scaler.transform(X_physico_te_raw)

# 6) Enhanced Dataset and Dataloader
class PMHCDataset(Dataset):
    def __init__(self, pep, physico, dist, labels):
        self.pep = torch.from_numpy(pep).float()
        self.physico = torch.from_numpy(physico).float()
        self.dist = torch.from_numpy(dist).float()
        self.labels = torch.from_numpy(labels).long()
    def __len__(self): return len(self.labels)
    def __getitem__(self, idx):
        return {
            "peptide": self.pep[idx],
            "physicochemical": self.physico[idx],
            "distances": self.dist[idx],
            "label": self.labels[idx]
        }

train_loader = DataLoader(PMHCDataset(X_pep_tr, X_physico_tr, X_dist_tr, y_tr), batch_size=32, shuffle=True)
val_loader = DataLoader(PMHCDataset(X_pep_te, X_physico_te, X_dist_te, y_te), batch_size=32)

# 7) Enhanced Model definition
class PMHCStabilityModel(nn.Module):
    def __init__(self, pep_dim, physico_dim, dist_dim, hidden=64, pos_emb_dim=8, num_classes=3):
        super().__init__()
        self.pos_emb = nn.Embedding(9, pos_emb_dim)
        pep_in = pep_dim + 9 * pos_emb_dim
        
        # Separate networks for each feature type
        self.pep_net = nn.Sequential(nn.Linear(pep_in, hidden), nn.ReLU(), nn.Dropout(0.3))
        self.physico_net = nn.Sequential(nn.Linear(physico_dim, hidden//2), nn.ReLU(), nn.Dropout(0.3))
        self.dist_net = nn.Sequential(nn.Linear(dist_dim, hidden), nn.ReLU(), nn.Dropout(0.3))
        
        # Combined classifier
        combined_dim = hidden + hidden//2 + hidden  # pep + physico + dist
        self.head = nn.Sequential(
            nn.Linear(combined_dim, hidden), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(hidden, num_classes)
        )

    def forward(self, pep, physico, dist):
        bsz = pep.size(0)
        pos_ids = torch.arange(9, device=pep.device).repeat(bsz, 1)
        pos_feat = self.pos_emb(pos_ids).view(bsz, -1)
        pep = torch.cat([pep, pos_feat], dim=1)
        
        # Process each feature type
        pep_out = self.pep_net(pep)
        physico_out = self.physico_net(physico)
        dist_out = self.dist_net(dist)
        
        # Combine all features
        combined = torch.cat([pep_out, physico_out, dist_out], dim=1)
        return self.head(combined)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PMHCStabilityModel(X_pep_tr.shape[1], X_physico_tr.shape[1], X_dist_tr.shape[1]).to(device)

print(f"Model created with:")
print(f"- Peptide features: {X_pep_tr.shape[1]}")
print(f"- Physicochemical features: {X_physico_tr.shape[1]}")
print(f"- Distance features: {X_dist_tr.shape[1]}")

# 8) Training loop
criterion = nn.CrossEntropyLoss(weight=get_class_weights(y_tr).to(device))
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=5)
patience, best_val, wait = 15, float("inf"), 0

for epoch in range(1, 201):
    model.train(); running = 0
    for batch in train_loader:
        optimizer.zero_grad()
        logits = model(
            batch["peptide"].to(device), 
            batch["physicochemical"].to(device),
            batch["distances"].to(device)
        )
        loss = criterion(logits, batch["label"].to(device))
        loss.backward(); optimizer.step()
        running += loss.item()
    train_loss = running / len(train_loader)

    model.eval(); val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for batch in val_loader:
            logits = model(
                batch["peptide"].to(device), 
                batch["physicochemical"].to(device),
                batch["distances"].to(device)
            )
            loss = criterion(logits, batch["label"].to(device))
            val_loss += loss.item()
            preds = logits.argmax(1).cpu()
            correct += (preds == batch["label"]).sum().item()
            total += batch["label"].size(0)
    val_loss /= len(val_loader)
    val_acc = correct / total * 100
    print(f"Epoch {epoch:3d} | train {train_loss:.4f} | val {val_loss:.4f} | acc {val_acc:.2f}%")

    scheduler.step(val_loss)
    if val_loss < best_val:
        best_val, wait = val_loss, 0
        torch.save(model.state_dict(), "best_pmhc_enhanced.pt")
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping! Best val loss:", best_val)
            break

# 9) Evaluation: get predicted class + probability
model.load_state_dict(torch.load("best_pmhc_enhanced.pt"))
model.eval()

predicted_labels = []
dl_scores = []
peptide_seqs_final = df['peptide_seq'].iloc[X_pep_te.shape[0] * -1:].tolist()  # assumes 20% test

target_class_index = 1  # "Moderately Stable"
class_map = {0: "Unstable", 1: "Moderately Stable", 2: "Highly stable"}

with torch.no_grad():
    for batch in val_loader:
        logits = model(
            batch["peptide"].to(device), 
            batch["physicochemical"].to(device),
            batch["distances"].to(device)
        )
        probs = torch.softmax(logits, dim=1).cpu().numpy()
        preds = probs.argmax(axis=1)

        predicted_labels.extend([class_map[i] for i in preds])
        dl_scores.extend(probs[:, target_class_index])

# Clip peptide list if mismatch
min_len = min(len(predicted_labels), len(peptide_seqs_final))

# Add physicochemical properties to output
physico_props_test = []
for seq in peptide_seqs_final[:min_len]:
    analysis = ProteinAnalysis(seq)
    props = {
        'molecular_weight': analysis.molecular_weight(),
        'charge_pH7': analysis.charge_at_pH(7.0),
        'hydrophobicity_KD': sum(hydrophobicity_scale.get(aa, 0) for aa in seq) / len(seq),
        'gravy': analysis.gravy(),
        'instability_index': analysis.instability_index(),
        'isoelectric_point': analysis.isoelectric_point()
    }
    physico_props_test.append(props)

physico_df_test = pd.DataFrame(physico_props_test)

dl_df = pd.DataFrame({
    'peptide_sequences': peptide_seqs_final[:min_len],
    'predicted_stability': predicted_labels[:min_len],
    'dl_score': dl_scores[:min_len]
})

# Combine with physicochemical properties
dl_df = pd.concat([dl_df, physico_df_test], axis=1)

dl_df.to_csv("dl_predictions.csv", index=False)
print("Deep Learning predictions with physicochemical properties saved to dl_predictions.csv")

# 10) ROC Curve Analysis
from sklearn.metrics import roc_curve, auc

# Binary label oluştur: Stable (>=12 dk) → 1, Unstable (<12 dk) → 0
df['binary_label'] = df['half_life'].apply(lambda x: 1 if x >= 12 else 0)

# ROC eğrisi için test verisine karşılık gelen binary label'ları al
true_labels_binary = df.set_index('peptide_seq').loc[peptide_seqs_final[:min_len], 'binary_label'].values

# ROC ve AUROC hesapla
fpr, tpr, _ = roc_curve(true_labels_binary, dl_scores[:min_len])
roc_auc = auc(fpr, tpr)

# ROC eğrisini çiz
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='seagreen', lw=2, label=f'Deep Learning (AUROC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Deep Learning ROC Curve')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Re-import required libraries after reupload
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load the uploaded files
stabpan_df = pd.read_csv("/content/2021639_NetMHCstabpan.csv", sep="\t")
true_df = pd.read_csv("/content/peptideseq_halflife.csv")

# Convert experimental half-life to stability class
true_df['true_stability'] = true_df['half_life'].apply(
    lambda x: "Unstable" if x < 12 else "Moderately Stable" if x < 2500 else "Highly stable"
)

# Also create binary label: 0 = unstable, 1 = stable (for ROC)
true_df['binary_label'] = true_df['half_life'].apply(lambda x: 1 if x >= 12 else 0)

# Convert NetMHCStabpan half-lives from hours to minutes
stabpan_df['half_life_min'] = stabpan_df['half_lives'] * 60
stabpan_df['predicted_stability'] = stabpan_df['half_life_min'].apply(
    lambda x: "Unstable" if x < 12 else "Moderately Stable" if x < 2500 else "Highly stable"
)

# Merge on peptide_sequences
merged_eval_df = pd.merge(true_df, stabpan_df, on="peptide_sequences")

# Prepare labels
y_true = merged_eval_df['true_stability']
y_pred = merged_eval_df['predicted_stability']
labels = ["Highly stable", "Moderately Stable", "Unstable"]

# Compute evaluation metrics
stabpan_eval_report = classification_report(y_true, y_pred, labels=labels, output_dict=True)
stabpan_eval_matrix = confusion_matrix(y_true, y_pred, labels=labels)
stabpan_eval_report_df = pd.DataFrame(stabpan_eval_report).transpose()

# Plot confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(stabpan_eval_matrix, annot=True, fmt='d',
            xticklabels=labels, yticklabels=labels, cmap="Purples")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix: NetMHCStabpan vs Experimental")
plt.tight_layout()
plt.show()

# Display classification report
print(stabpan_eval_report_df)
stabpan_eval_report_df.to_csv("netmhcstabpan_vs_experimental_report.csv")

# Assign score for ROC (based on class confidence approximation)
# Highly stable = 1.0, Moderately Stable = 0.5, Unstable = 0.0
stabpan_score_map = {
    "Highly stable": 1.0,
    "Moderately Stable": 0.5,
    "Unstable": 0.0
}
merged_eval_df['sp_score'] = merged_eval_df['predicted_stability'].map(stabpan_score_map)

# Save merged predictions for ROC use
stabpan_predictions_df = merged_eval_df[['peptide_sequences', 'true_stability', 'predicted_stability', 'binary_label', 'sp_score']]
stabpan_predictions_df.to_csv("netmhcstabpan_predictions.csv", index=False)
print("Predictions saved to netmhcstabpan_predictions.csv")


In [ ]:
# COMPARISON OF ALL MODELS
import pandas as pd
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Load prediction outputs
rf_df = pd.read_csv("rf_predictions.csv")
dl_df = pd.read_csv("dl_predictions.csv")
sp_df = pd.read_csv("netmhcstabpan_predictions.csv")
true_df = pd.read_csv("peptideseq_halflife.csv")

# Create binary label from true data
true_df['binary_label'] = true_df['half_life'].apply(lambda x: 1 if x >= 12 else 0)

# Add half_life_min to NetMHCStabpan predictions for ROC scoring
netmhc_raw = pd.read_csv("2021639_NetMHCstabpan.csv", sep="\t")
sp_df = pd.merge(sp_df, netmhc_raw[['peptide_sequences', 'half_lives']], on="peptide_sequences", how="left")
sp_df['half_life_min'] = sp_df['half_lives'] * 60

# Merge all on peptide_sequences
merged = rf_df.merge(dl_df, on="peptide_sequences", suffixes=('_rf', '_dl'))
merged = merged.merge(sp_df[['peptide_sequences', 'half_life_min']], on="peptide_sequences")
merged = merged.merge(true_df[['peptide_sequences', 'binary_label']], on="peptide_sequences")

print(" Merged dataset shape:", merged.shape)

# Compute ROC curves using true probability scores
fpr_rf, tpr_rf, _ = roc_curve(merged['binary_label'], merged['rf_score'])
fpr_dl, tpr_dl, _ = roc_curve(merged['binary_label'], merged['dl_score'])
fpr_sp, tpr_sp, _ = roc_curve(merged['binary_label'], merged['half_life_min'])

# Compute AUC
auc_rf = auc(fpr_rf, tpr_rf)
auc_dl = auc(fpr_dl, tpr_dl)
auc_sp = auc(fpr_sp, tpr_sp)

# Plot ROC curves
plt.figure(figsize=(8, 6))
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUROC = {auc_rf:.2f})', color='royalblue')
plt.plot(fpr_dl, tpr_dl, label=f'Deep Learning (AUROC = {auc_dl:.2f})', color='seagreen')
plt.plot(fpr_sp, tpr_sp, label=f'NetMHCStabpan (AUROC = {auc_sp:.2f})', color='darkorange')
plt.plot([0, 1], [0, 1], '--', color='gray')

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison (All Using Real Scores)")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()
